# Trabajo Final procesamiento de lenguaje natural (NLP) <a class="tocSkip">
## Universidad Pontificia Bolivariana <a class="tocSkip">

**Estudiante:** Jeronimo Valencia Ospina

**ID:** 000165785



**Subir Este notebook de Trabajo Final en su repositorio de GitHub. y enviar el link en **Microsoft TEAMS**

Docente: [Jose R. Zapata](https://joserzapata.github.io)
- https://joserzapata.github.io
- https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/       


## Objetivo del Trabajo
Realizar un proceso de Procesamiento de lenguaje natural (NLP) para dejar los datos preparados para ser usados con algoritmos de Machine Learning para Clasificación como objetivo final del trabajo.

los datos preparados para ser usados con algoritmos de Machine Learning para Regresión o Clasificación como objetivo final del trabajo.

El trabajo se realizara en este jupyter notebook y subirlo a su repositorio de github creado en clase. (**Recuerde poner su nombre e información**)

## Las actividades a realizar
    

1) Limpiar los datos de texto.

   - https://joserzapata.github.io/courses/nlp/procesamiento-basico/

   - https://joserzapata.github.io/courses/nlp/preprocesamiento-texto/

2) Realizar la representación de texto:

   - https://joserzapata.github.io/courses/nlp/representaciones/

   - Tokenización
   - Lematización o stemming
   - Representación de los datos de texto (Bag of Words o TF-IDF)

3) Utilice un modelo de Machine Learning para clasificación para entrenar y evaluar el modelo.
        
    - Calcule estas las métricas de evaluación (accuracy, precision, recall, f1-score) - https://joserzapata.github.io/courses/python-ciencia-datos/clasificacion/#evaluacion-modelo-simple


    
   
    
*NOTA: No dude en contactarme para cualquier pregunta o inquietud :) por el chat de Teams o al correo
joser.zapata@upb.edu.co*

## EVALUACIÓN


|Porcentaje en la evaluación | Descripción| Nada | Incompleto | Completo 
| :---: |:---: |:---: |:---: |:---: 
| 5 % |**limpieza de los datos** |
| 5 % | **Representacion de los datos** |
| 5 % | **Machine Learning** <br> Entrenar y evaluar el modelo propuesto |


In [30]:
import pandas as pd
import numpy as np

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    classification_report,
)
from sklearn.model_selection import train_test_split

Carga del dataset

El archivo  se carga en aproximadamente 10 segundos

In [31]:
# Usar su ID UPB Ejemplo: "0028984798"
id_upb = "000165785"

data_reviews = pd.read_parquet(
    "https://www.dropbox.com/scl/fi/gvk9yj8cn96oocr9z058x/filmaffinity_reviews_dataset.parquet?rlkey=xgvr00zvkxbkwqqavqutpsshg&st=xjb7xze9&dl=1"
)
data_reviews = data_reviews.sample(n=50_000, random_state=int(id_upb))
data_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 50000 entries, 132597 to 125483
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   author_review_desc  50000 non-null  string
 1   author_rating       50000 non-null  int64 
dtypes: int64(1), string(1)
memory usage: 1.1 MB


Ejemplo de algunas filas del dataset

In [32]:
data_reviews.sample(5)

,author_review_desc,author_rating
53374,\nFallida tragicomedia envuelta en una estétic...,4
114333,\nMe ha gustado mucho esta película. Estuve de...,9
53332,"\nHay maridajes que, aparentemente imposibles,...",5
30967,\nSi te gusta el género de aventuras es una pe...,8
60340,\nCreo que el TDAH y el uso de drogas de la fo...,3


Evaluar los valores nulos

In [33]:
data_reviews.isnull().sum()

author_review_desc    0
author_rating         0
dtype: int64

En este caso no hay nulos.

Revisar si hay valores duplicados:

In [34]:
data_reviews.duplicated().sum()

np.int64(2)

No hay  valores duplicados.

## Clasificación Tradicional para Análisis de Sentimientos y Categorías 👍 👎



Definición: Se considera reseña positiva cuando la puntuación ("author_rating") es mayor que 6; negativa en caso contrario.

Crear la variable binaria de sentimiento: 1 (positivo) si author_rating > 6, 0 (negativo) de lo contrario

In [35]:
UMBRAL = 6
data_reviews["sentiment_bin"] = (data_reviews["author_rating"] > UMBRAL).astype(int)

In [36]:
# Dividir los datos en conjuntos de entrenamiento y prueba

X_data = data_reviews["author_review_desc"]
y_data = data_reviews["sentiment_bin"]


Dividir el dataset en entrenamiento y prueba

In [37]:
X_train, X_test, y_train, y_test = train_test_split(
    X_data,
    y_data,
    test_size=0.2,
    stratify=y_data,  # Mantener la proporción de clases en ambos conjuntos
    random_state=42,
)

## Análisis exploratorio de los datos

para determinar que tipo de limpieza se debe realizar a los datos de texto de `X_train`

In [38]:
X_train.info()

<class 'pandas.core.series.Series'>
Index: 40000 entries, 146168 to 62870
Series name: author_review_desc
Non-Null Count  Dtype 
--------------  ----- 
40000 non-null  string
dtypes: string(1)
memory usage: 625.0 KB


In [39]:
X_train.head()

146168    \nAquí con Ben Stiller te ríes, al principio c...
2324      \nPareciera que el escenario que se plantea no...
139624    \nDocumentalmente y cinematográficamente, está...
119957    \nLa pongo un 5 por el preciso análisis que ha...
30878     \nEmotiva y tierna película dirigida por el gr...
Name: author_review_desc, dtype: string

Se deben realizar transformaciones de texto para normalizarlos y se pueda realizar el aprendizaje de manera mas efectiva

## 1) Limpieza de los datos de texto

Tomar los datos de `X_train` y aplicar las funciones de limpieza que considere necesarias

Ayudas:

- Convertir a minúsculas
- Eliminar caracteres especiales y números
- Corregir palabras mal escritas
- etc


In [40]:
X_train.isnull().sum()

np.int64(0)

In [41]:
import re
import string

In [42]:
def limpiar_texto(texto):
    # Convertir a minúsculas
    texto = str(texto).lower()

    MAP_VOCALES = {
        'á': 'a',
        'é': 'e',
        'í': 'i',
        'ó': 'o',
        'ú': 'u',
        'ü': 'u'
    }
    #Eliminar tildes
    quit_tilde = str.maketrans(MAP_VOCALES)
    texto = texto.translate(quit_tilde)

    # Eliminar textos entre corchetes
    texto = re.sub(r"\[.*?\]", "", texto)

    # Eliminar etiquetas HTML
    texto = re.sub(r"<.*?>+", "", texto)

    # Eliminar URLs
    texto = re.sub(r'http\S+|www\S+|https\S+', '', texto, flags=re.MULTILINE)

    # Eliminar puntuación
    texto = texto.translate(str.maketrans('', '', string.punctuation))

    # Eliminar saltos de línea
    texto = re.sub(r"\n", " ", texto)

    # Eliminar números
    texto = re.sub(r"\w*\d\w*", "", texto)

    # Eliminar emojis y caracteres especiales
    texto = re.sub(r"[^\x00-\x7Fñ]+", "", texto)

    # Eliminar espacios extras entre palabras
    texto = re.sub(r"\s+", " ", texto)

    # Eliminar espacios en blanco adicionales
    texto = texto.strip()

    return texto

Probar función

In [43]:
text_ejemplo = "¡Hola! Feliz año nuevo. Visita https://example.com para más info. é í ó ú ñ ü"

limpiar_texto(text_ejemplo)

'hola feliz año nuevo visita para mas info e i o u ñ u'

In [44]:
X_train = X_train.apply(limpiar_texto)
X_train.head()

146168    aqui con ben stiller te ries al principio crea...
2324      pareciera que el escenario que se plantea no e...
139624    documentalmente y cinematograficamente esta de...
119957    la pongo un por el preciso analisis que hace d...
30878     emotiva y tierna pelicula dirigida por el gran...
Name: author_review_desc, dtype: object

## 2) Representacion del texto

Luego de tener los datos limpios, realizar la representación de los datos de texto para poder usarse en modelos de machine learning.

### Tokenización

In [45]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
nltk.download("punkt_tab")

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /home/jervalencia/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [46]:
sample_txt = X_train.iloc[0]
print("Texto de ejemplo:", sample_txt)

Texto de ejemplo: aqui con ben stiller te ries al principio crean un ambiente creible y natural pero flojea en el segundo tramo de la pelicula que se vuelve muy romanticona americanoide hollywoodiense y estereotipica le querian dar el sabor de los insulsos y superfluos años a la pelicula original y he ahi el resultado un producto con poco discurso que dar y que otras obras han desarrollado de una manera mas ingeniosa y brillante


In [47]:
tokens_nltk = word_tokenize(sample_txt, language="spanish")
sentences_nltk = sent_tokenize(sample_txt, language="spanish")

In [48]:
from nltk.corpus import stopwords

In [49]:
nltk.download("stopwords")
stopword_es = set(stopwords.words("spanish"))

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jervalencia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [50]:
tokens_no_stop = [word for word in tokens_nltk if word not in stopword_es]
print("Tokens originales:", tokens_nltk)
print("Tokens sin stopwords:", tokens_no_stop)

Tokens originales: ['aqui', 'con', 'ben', 'stiller', 'te', 'ries', 'al', 'principio', 'crean', 'un', 'ambiente', 'creible', 'y', 'natural', 'pero', 'flojea', 'en', 'el', 'segundo', 'tramo', 'de', 'la', 'pelicula', 'que', 'se', 'vuelve', 'muy', 'romanticona', 'americanoide', 'hollywoodiense', 'y', 'estereotipica', 'le', 'querian', 'dar', 'el', 'sabor', 'de', 'los', 'insulsos', 'y', 'superfluos', 'años', 'a', 'la', 'pelicula', 'original', 'y', 'he', 'ahi', 'el', 'resultado', 'un', 'producto', 'con', 'poco', 'discurso', 'que', 'dar', 'y', 'que', 'otras', 'obras', 'han', 'desarrollado', 'de', 'una', 'manera', 'mas', 'ingeniosa', 'y', 'brillante']
Tokens sin stopwords: ['aqui', 'ben', 'stiller', 'ries', 'principio', 'crean', 'ambiente', 'creible', 'natural', 'flojea', 'segundo', 'tramo', 'pelicula', 'vuelve', 'romanticona', 'americanoide', 'hollywoodiense', 'estereotipica', 'querian', 'dar', 'sabor', 'insulsos', 'superfluos', 'años', 'pelicula', 'original', 'ahi', 'resultado', 'producto', '

### Stemming

In [51]:
from nltk.stem import SnowballStemmer

sample_txt = X_train.iloc[0]
print("Texto de ejemplo:", sample_txt)

tokens = word_tokenize(sample_txt, language="spanish")

print("Tokens:", tokens)

Texto de ejemplo: aqui con ben stiller te ries al principio crean un ambiente creible y natural pero flojea en el segundo tramo de la pelicula que se vuelve muy romanticona americanoide hollywoodiense y estereotipica le querian dar el sabor de los insulsos y superfluos años a la pelicula original y he ahi el resultado un producto con poco discurso que dar y que otras obras han desarrollado de una manera mas ingeniosa y brillante
Tokens: ['aqui', 'con', 'ben', 'stiller', 'te', 'ries', 'al', 'principio', 'crean', 'un', 'ambiente', 'creible', 'y', 'natural', 'pero', 'flojea', 'en', 'el', 'segundo', 'tramo', 'de', 'la', 'pelicula', 'que', 'se', 'vuelve', 'muy', 'romanticona', 'americanoide', 'hollywoodiense', 'y', 'estereotipica', 'le', 'querian', 'dar', 'el', 'sabor', 'de', 'los', 'insulsos', 'y', 'superfluos', 'años', 'a', 'la', 'pelicula', 'original', 'y', 'he', 'ahi', 'el', 'resultado', 'un', 'producto', 'con', 'poco', 'discurso', 'que', 'dar', 'y', 'que', 'otras', 'obras', 'han', 'des

In [52]:
import spacy
nlp_es = spacy.load("es_core_news_sm")

In [53]:
stemmer = SnowballStemmer("spanish")

stemmed_tokens = [stemmer.stem(token) for token in tokens_no_stop]
print("Tokens después de stemming:", stemmed_tokens)

doc_es = nlp_es(sample_txt)
lemmatized_tokens = [token.lemma_ for token in doc_es if token.text.lower() not in stopword_es]
print("Tokens después de lematización (sin stopwords):", lemmatized_tokens)

Tokens después de stemming: ['aqui', 'ben', 'still', 'ries', 'principi', 'cre', 'ambient', 'creibl', 'natural', 'floje', 'segund', 'tram', 'pelicul', 'vuelv', 'romanticon', 'americanoid', 'hollywoodiens', 'estereotip', 'queri', 'dar', 'sabor', 'insuls', 'superflu', 'años', 'pelicul', 'original', 'ahi', 'result', 'product', 'discurs', 'dar', 'obras', 'desarroll', 'maner', 'mas', 'ingeni', 'brillant']
Tokens después de lematización (sin stopwords): ['aqui', 'ben', 'stiller', 'riser', 'principio', 'crear', 'ambiente', 'creible', 'natural', 'flojea', 'segundo', 'tramo', 'pelicula', 'volver', 'romanticona', 'americanoidir', 'hollywoodiense', 'estereotipico', 'queriar', 'dar', 'sabor', 'insulso', 'superfluo', 'año', 'pelicula', 'original', 'ahi', 'resultado', 'producto', 'discurso', 'dar', 'obra', 'desarrollar', 'manera', 'mas', 'ingeniós', 'brillante']


Representación Bag-Of-Words con CountVectorizer

In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

corpus = X_train.tolist()

cv = CountVectorizer()
bow_matrix = cv.fit_transform(corpus)

print("Dimensiones de la matriz BoW:", bow_matrix.shape)
print("Ejemplos de términos:", np.random.choice(cv.get_feature_names_out(), 10, replace=False))

Dimensiones de la matriz BoW: (40000, 144446)
Ejemplos de términos: ['bienvenidos' 'taku' 'presentimos' 'yimou' 'laguerta' 'huesped' 'iang'
 'concluyese' 'aplaudidos' 'vicisitudes']


Representación TF-IDF con TfidfVectorizer

In [55]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(corpus)

print("Dimensiones de la matriz TF-IDF:", tfidf_matrix.shape)
print("Ejemplos de términos:", np.random.choice(tfidf.get_feature_names_out(), 10, replace=False))

Dimensiones de la matriz TF-IDF: (40000, 144446)
Ejemplos de términos: ['autoplagio' 'compartimos' 'dacosta' 'esteticista' 'abducted' 'acercaos'
 'triet' 'lechera' 'tenacidad' 'decente']


In [56]:
feature_names = tfidf.get_feature_names_out()
mean_tfidf = np.asarray(tfidf_matrix.mean(axis=0)).ravel()

total = mean_tfidf.sum()
probs = mean_tfidf / total if total > 0 else np.zeros_like(mean_tfidf)

df_terms = pd.DataFrame({
    "term": feature_names,
    "mean_tfidf": mean_tfidf,
    "probability": probs
})

print("Top 10 términos más importantes según TF-IDF:")
print(df_terms.sample(10).sort_values("probability", ascending=False).reset_index(drop=True))

Top 10 términos más importantes según TF-IDF:
           term  mean_tfidf   probability
0        gozosa    0.000032  3.298404e-06
1      pedofila    0.000009  9.294107e-07
2  sirviendonos    0.000005  5.649537e-07
3      princese    0.000005  5.595237e-07
4          yuko    0.000005  4.696409e-07
5    trastocara    0.000005  4.677800e-07
6  reflecciones    0.000004  3.800762e-07
7      callback    0.000004  3.773112e-07
8     jubilarlo    0.000003  3.594433e-07
9      logtando    0.000002  2.392978e-07


## 3) Entrenar un modelo de machine learning de clasificación

Utilizar un modelo de clasificación para entrenar y evaluar el modelo con los datos preparados.


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

In [58]:
model = LogisticRegression(penalty='l2', solver='liblinear', C=1.0)

logistic_complete = Pipeline([
    ("tfidf", TfidfVectorizer()),
    ("classification", LogisticRegression(
        penalty='l2',
        solver='liblinear',
        C=1.0,
        random_state=42
    ))
])

logistic_complete.fit(X_train, y_train)

,steps,"[('tfidf', ...), ('classification', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [59]:
X_test = X_test.apply(limpiar_texto)

In [60]:
y_pred = logistic_complete.predict(X_test)

### Evaluar el modelo con los datos de prueba

Usar el modelo para predecir en `X_test` y evaluar con `y_test`

**Nota:** Recuerde que `X_test` debe pasar por los mismos procesos de limpieza y representación que `X_train`

In [61]:
def summarize_classification(y_test, y_pred):

    acc = accuracy_score(y_test, y_pred, normalize=True)
    prec = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc = roc_auc_score(y_test, y_pred)

    return {'accuracy': acc,
            'precision': prec,
            'recall':recall,
            'f1':f1,
            'roc':roc
            }

In [62]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1 Score:", f1_score(y_test, y_pred))
print("ROC AUC Score:", roc_auc_score(y_test, y_pred))
print("Reporte de clasificación:\n", classification_report(y_test, y_pred))

Accuracy: 0.8063
Precision: 0.8054964539007092
Recall: 0.8439531859557867
F1 Score: 0.8242765127460764
ROC AUC Score: 0.8031765063415495
Reporte de clasificación:
               precision    recall  f1-score   support

           0       0.81      0.76      0.78      4617
           1       0.81      0.84      0.82      5383

    accuracy                           0.81     10000
   macro avg       0.81      0.80      0.80     10000
weighted avg       0.81      0.81      0.81     10000



**Guardar el modelo**

In [63]:
import joblib

dump_path = "logistic_sentiment_model.joblib"
joblib.dump(logistic_complete, dump_path)
print(f"Modelo guardado en: {dump_path}")

Modelo guardado en: logistic_sentiment_model.joblib


## Librerías Usadas

In [64]:
from watermark import watermark

print(watermark(python=True, iversions=True, globals_=globals()))

Python implementation: CPython
Python version       : 3.12.3
IPython version      : 9.6.0

pandas   : 2.3.3
watermark: 2.5.0
re       : 2.2.1
numpy    : 2.3.4
nltk     : 3.9.2
joblib   : 1.5.2
spacy    : 3.8.7
sklearn  : 1.7.2



---

## Referencias
- [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
- [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html)
- [Classification of text documents using sparse features](https://scikit-learn.org/stable/auto_examples/text/plot_document_classification_20newsgroups.html)
- Ejemplo de Entrenamiento y selección de Modelo de machine learning entre varios modelos <https://joserzapata.github.io/post/ciencia-datos-proyecto-python/6-model_selection/>
- https://joserzapata.github.io/courses/python-ciencia-datos/python/
- https://joserzapata.github.io/courses/python-ciencia-datos/pandas/
- https://joserzapata.github.io/courses/python-ciencia-datos/machine-learning/

Docente: [Jose R. Zapata](https://joserzapata.github.io)

- [https://joserzapata.github.io/](https://joserzapata.github.io/)
- [https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/](https://www.linkedin.com/in/jose-ricardo-zapata-gonzalez/)